# Modelling the Word Game Wordle

The following notebook was created by Ken Pierce. The intention here give some practice in identifying and including pre-conditions, post-conditions, and data invariants using assertions in Python. These are fundamental concepts in creation of formal models, and the use of Python assertions allows us to explore them in a familiar setting.

## Wordle

Wordle is a word game created by Josh Wardle (https://twitter.com/powerlanguish); it is now owned and published by the New York Times: https://www.nytimes.com/games/wordle/index.html. The instructions for Wordle are as follows:

* Guess the WORDLE in 6 tries.
* Each guess must be a valid 5 letter word. 
* The color of the tiles will change to show how close your guess was to the word.
* A green letter shows it is in the correct position.
* A yellow letter shows it is in the word but in the wrong position.
* A grey letter shows that it is not in the word in any position.

## Building Models

There is no right or wrong way to construct a programme (or formal specification) from natural language specifications. We need to consider the purpose, which guides the abstraction decisions. The steps however can be broadly broken down as:

1. Analyse the functional behaviour from the requirements
2. Extract a list of possible data types (often from nouns) and functions (often from actions)
3. Create a dictionary by giving explanations to items in the list
4. Sketch out data types
5. Sketch out functions
6. Refine and add restrictions
7. Review and refine

The following will guide us through these steps to define a Wordle game.

## Functional Behaviour and Data Types

From the description above, we can identify the key elements of the instructions to understand the functionality and suggest data types:

* There’s a secret **wordle**
    - It has **five** letters (we can infer this though it is not explicit)
    - The player loses after **six** tries
* The player can make a **guess**
    - Guess must be **five** letters
    - Guess must be **valid** (= real word?)
* The game should output for each letter in a guess:
    - If the letter is in the **correct position**
    - If the letter in the **wrong position** (but in the word)
    - If the letter is **not in the word**

# The Programme

## Setup

Here, we simply include some imports for the later, as well as download a list of valid 5-letter words. Note, if the download  does not work, you can simply define a list of custom list of 5-letter strings, i.e. `WORDS = ["HELLO", ...]`.

In [7]:
from dataclasses import dataclass   # for defining dataclasses
from enum import Enum               # for defining enumerations
import typing                       # for type hinting
import random                       # for selecting a random answer
from typing import List             # for defining custom typed lists
from urllib.request import urlopen  # to load web data

# grab a list of words
url = 'https://raw.githubusercontent.com/tabatkins/wordle-list/main/words'
WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url).readlines()]



## Data Types and Contants

Next, we can define some simple data types, and constants that we will need to use, based on the above analysis. Here we define:

* The length of a valid word.
* The maximum number of guesses.
* A `Word` type as an alias of `str`
* An enumeration of the three types of clue (i.e. colours)
* An enumeration for the game state, either in play, won, or lost.

In [8]:
# constants
WORD_LENGTH = 5 
MAX_GUESSES = 6

# define Word as an alias for string
Word = str

# type enumerating the three possible clue colours
Clue = Enum('Clue', ['GREEN', 'YELLOW', 'GREY'])

# type enumerating the state of the game
Gamestate =  Enum('Gamestate', ['WON', 'LOST', 'PLAYING'])

## Guess

We can now introduce a `Guess` type, which represents a guess after it is processed by the game (i.e. one line of the output). We use the `dataclass' decorator and properties to simplify definition. This includes the word that was guessed, and the clues.

1. There are two invariants to add to the `setter` functions; what are these? Complete the assertions and include an appropriate message for when they fail. These should restrict the values that `word` and `clues` can take. 
2. Create some test values for `Guess` to demonstrate the invariants, i.e.  `g1 = Guess("HELLO", [Clue.YELLOW, ...])`.

In [9]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        # invariant
        assert len(word) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.word) <> {WORD_LENGTH}"  
        self._word = word

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        # invariant
        assert len(clues) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.clues) <> {WORD_LENGTH}"              
        self._clues = clues

    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"       


# Auxiliary Functions

There are multiple ways to specify the game. The suggestion here is to have a main class `Game` that includes the state and top-level functions including `make_guess`. To help define those, we define two auxiliary functions:

* `check_letter`: Compute clue given a character, its position, and word
* `check_guess`: Compute clues given a word and the answer

## check_letter 

This function computes the clue (colour) for a given letter, given a word and the index of the letter in the word. We will start with a naïve implementation:

* If it's in the right place, return green (i.e. `Clue.GREEN`)
* If it’s not there, return grey
* Otherwise, return yellow.

Complete the function in the following way:

3. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
4. Complete the implementation based on the sketch above.
5. Include some test cases to check your implementation, e.g. `print(check_letter("S", 0, "STOUT"))`.

In [10]:
def check_letter(letter: str, index: int, word: Word) -> Clue:
    """
    Given a letter and an index, computes the colour of the blue
    based on the word.
    """
    # pre-condition
    assert 0 <= index < WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_letter failed."
           
    if word[index] == letter: return Clue.GREEN  
    elif letter not in word: return Clue.GREY
    else: return Clue.YELLOW

## Check Guess

Given a word and the correct answer, this function computes the list of clues. This can be achieved by declaring a local variable, and for each letter in the guess, use `check_letter`; we can use `enumerate(guess)` to get the index.

6. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
7. Complete the implementation based on the sketch above.
8. Include some test cases to check your implementation, e.g. `print(check_guess("STAND", "STOUT"))`.

In [14]:

def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    # pre-condition
    assert len(word) == WORD_LENGTH and \
           len(guess) == WORD_LENGTH, "pre-check_guess failed"
           
    clues = []
    for i,letter in enumerate(guess):
        clue = check_letter(letter, i ,word)
        clues.append(clue)
    return clues

print(check_guess("STAND", "STOUT"))

[<Clue.GREEN: 1>, <Clue.GREEN: 1>, <Clue.GREY: 3>, <Clue.GREY: 3>, <Clue.YELLOW: 2>]


# Game State and Top-level Functionality

Here we define a `Game` class; the state is the current wordle (`answer`), the guesses made so far, and the state of the game (playing, won, or lost). Functionlity is included to print a message to the player about the game, and to rest the game when it is over.

## make_guess

The top-level function is `make_guesss` which takes a single word as a parameter. This function should use `check_guesss` to compute the guess and store it in `guesses`; it should then update the game state by checking if the game has been won (the word is exactly the same as the answer), or lost (the player reached the maximum number of guesses).

9. What pre-conditions should be included? These should restrict the parameters and the game state in which a guess is made (e.g. the player should not be able to guess if the game finished). Add these and an appropriate message.
10. Complete the implementation based on the sketch above, and play a game! You can alternate `game.make_guess(...)` and `game.print_state()` to play.

Note, can you spot the problem with the naïve implementation of `check_letter`?

In [15]:

class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer="STOUT"):
        """
        Constructor for game.
        """
        self.answer = answer
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        # make guesses uppercase
        word = word.upper() 

        # pre-condition
        assert self.gstate == Gamestate.PLAYING and \
               len(self.guesses) < MAX_GUESSES and \
               word in WORDS, "pre-guess failed."
               
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Prints a message to the user based on the current state of the game.
        """
        for guess in self.guesses: print(guess)
        if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
        else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING
   
game = Game()
game.print_state()
game.make_guess("stand")


Guess the wordle, you have 6 guesses remaining.


# Changes Below

## Hint

Now that the base rules and basic gameplay have been established, we can continue to enhance the gameplay experience by providing the player with a hint - if they choose to receive one. As we are given that hint must be a single character between [A-Z], I will make the hint simply reveal a letter they have not tried yet that is currently in the word.


In [42]:
import re

Hint = str

def hint(guesses: List[Guess], answer: Word) -> Hint:
    #.word and .clues

    assert re.match('^[A-Z]{%d}$' % WORD_LENGTH, answer) and \
        game.gstate == Gamestate.PLAYING and \
        answer in WORDS and \
        len(guesses) < MAX_GUESSES, "pre-hint failed."
    
    greens = list(range(0, WORD_LENGTH))

    for i in guesses:
        for j in range(0, WORD_LENGTH):
            if i.clues[j] == Clue.GREEN:
                greens.remove(j)
    
    hint = answer[random.choice(greens)]

    assert re.match('^[A-Z]$', hint)

    return hint

print(hint(game.guesses, game.answer))

AssertionError: pre-hint failed.

## Testing

This section shows all the tests that have been made for each of the methods that require it, with all possible scenarios accounted for. The adjacent comment will describe the expected output of the test.

### Guess

In [35]:
g1 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW]) # Successful Guess object creation
print(g1) # HELLO: ['H: YELLOW', 'E: YELLOW', 'L: GREY', 'L: GREEN', 'O: YELLOW'] (using __repr__)

g2 = Guess("HELL", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW]) # Assertion Error
g3 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN]) # Assertion Error

HELLO: ['H: YELLOW', 'E: YELLOW', 'L: GREY', 'L: GREEN', 'O: YELLOW']


AssertionError: Invariant violated: len(Guess.word) <> 5

### check_letter

In [ ]:
print(check_letter("S", 0, "STOUT")) #Clue.GREEN
print(check_letter("S", 3, "STOUT")) #Clue.YELLOW
print(check_letter("Z", 0, "STOUT")) #Clue.GREY

### Hint

In [ ]:
    
print(hint(game.guesses, game.answer))